<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/bert_bloom_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import re
import nltk
import unicodedata
import nltk
import gensim
import math
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [5]:
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.8 MB/s eta 0:00:00


In [12]:
from docx import Document

# Load the document
doc_path = "/content/sample_data/Hair Document.docx"  # Update the path if necessary
doc = Document(doc_path)

# Extract text from the document
doc_text = ""
for paragraph in doc.paragraphs:
    doc_text += paragraph.text + "\n"

# Display the first few characters of the document text
print(doc_text[:200])


Hair has psychological and sociological importance throughout the ages in framing the personality and general appearance of an individual. Significant progress is being made on discovering an effectiv


In [13]:
import re

# Perform basic preprocessing: remove special characters and lower the text
doc_text_cleaned = re.sub(r'[^a-zA-Z\s]', '', doc_text).lower()

# Tokenize the text into sentences or passages
sentences = doc_text_cleaned.split('\n')

# Display the first few sentences
for sentence in sentences[:5]:
    print(sentence)


hair has psychological and sociological importance throughout the ages in framing the personality and general appearance of an individual significant progress is being made on discovering an effective and safe drug for hair growth angiogenesis androgen antagonism vasodilation potassium channel opening and alpha reductase inhibition are the major nonsurgical therapeutic strategies of hair growth promotion in spite of a flood of drugs claiming to be useful as hair growth promoters more rational strategies which can target the problem areas or stages of the hair growth cycle effectively are still awaited this article highlights the developments in hair rejuvenation strategies and reviews the potential of herbal drugs as safer and effective alternatives

hair has been a sign of beauty and a contribution to an individuals personality since time immemorial alopecia baldness a dermatological disorder is a common problem of cosmetology as well as primary health practice it is common throughout

In [15]:
# Load the question-answer pairs document
qna_pairs_path = "/content/sample_data/Hair qna pairs.docx"  # Update the path if necessary
qna_doc = Document(qna_pairs_path)

# Extract text from the question-answer pairs document
qna_doc_text = ""
for paragraph in qna_doc.paragraphs:
    qna_doc_text += paragraph.text + "\n"

# Perform basic preprocessing: remove special characters and lower the text
qna_doc_text_cleaned = re.sub(r'[^a-zA-Z0-9\s]', '', qna_doc_text).lower()

# Tokenize the text into question-answer pairs
qna_pairs = [pair.split('\n') for pair in qna_doc_text_cleaned.split('\n\n')]

# Display the first few question-answer pairs
for qna_pair in qna_pairs[:3]:
    print(qna_pair)


['hair question and answer pairs']
['question what are the major nonsurgical therapeutic strategies for promoting hair growth', 'answer the major nonsurgical therapeutic strategies for promoting hair growth include angiogenesis androgen antagonism vasodilation potassium channel opening and 5alpha reductase inhibition', 'question what is the significance of herbal drugs in hair growth promotion', 'answer herbal drugs have been considered safer and effective alternatives for promoting hair growth since ancient times and are widely used in ayurveda chinese medicine and unani systems', 'question what are the two us fdaapproved synthetic drugs for treating alopecia', 'answer minoxidil and finasteride are the two us fdaapproved synthetic drugs for treating alopecia although their side effects have limited their usage', 'question what are the phases of the hair growth cycle', 'answer the phases of the hair growth cycle are anagen growth catagen regression telogen resting and exogen shedding',

In [16]:
from transformers import BertForQuestionAnswering, GPT2LMHeadModel

# Load pre-trained BERT model for question-answering
bert_model_name = "bert-base-uncased"  # You can also use other variants if needed
bert_retriever = BertForQuestionAnswering.from_pretrained(bert_model_name)

# Load pre-trained BLOOM (GPT) model for text generation
bloom_model_name = "gpt2-medium"  # You can adjust the model size as needed
bloom_generator = GPT2LMHeadModel.from_pretrained(bloom_model_name)

# Display model configurations
print("BERT Model Configuration:")
print(bert_retriever.config)

print("\nBLOOM Model Configuration:")
print(bloom_generator.config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

BERT Model Configuration:
BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}


BLOOM Model Configuration:
GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type"

In [18]:
from tqdm import tqdm

In [20]:
tokenized_qna_pairs = []

for i, qna_pair in enumerate(tqdm(qna_pairs)):
    # Tokenize the current question-answer pair
    tokenized_qna_pairs.append(tokenizer.encode_plus(
        qna_pair,
        truncation=True,
        padding=True,
        return_tensors="pt",
    ))

100%|██████████| 21/21 [00:00<00:00, 2871.50it/s]


In [26]:


# Import the required modules
import torch
from transformers import BertForQuestionAnswering, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Define the input IDs and attention masks for the "Hair Document"
input_ids_doc = torch.tensor([1, 2, 3, 4, 5])  # Replace with actual input IDs
attention_masks_doc = torch.tensor([1, 1, 1, 1, 1])  # Replace with actual attention masks

# Create the TensorDataset for the "Hair Document"
dataset_doc = TensorDataset(input_ids_doc, attention_masks_doc)

# Create the DataLoader for the "Hair Document"
dataloader_doc = DataLoader(
    dataset_doc,
    sampler=RandomSampler(dataset_doc),
    batch_size=batch_size,
)
# Initialize the BERT model for question-answering
bert_model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define the optimizer
optimizer = AdamW(bert_model.parameters(), lr=learning_rate)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [30]:
import torch
from transformers import BertForQuestionAnswering
from torch.utils.data import DataLoader

In [31]:
dataloader_qna_pairs = DataLoader(...)
dataloader_doc = DataLoader(...)